In [1]:
# Bayesian with MCMC - Gibbs sampling
# Simple example: Inference parameter mu (the mean) of a multivariate Normal distribution
# Author: tran.vuduc[at]gmail.com
import numpy as np
import edward as ed
from edward.models import Empirical, Normal
import tensorflow as tf
import matplotlib.pyplot as plt

/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Model
n = 5 # Variable dimension
nobs = 30000 # number of observations
ns = 1000 # number of samples
mu = Normal(loc=tf.zeros(n), scale=tf.ones(n)) # This is the parameter to be estimated
x = Normal(loc=mu, scale=tf.ones(n), sample_shape=nobs) # This is the observation variable
qmu = Empirical(tf.Variable(tf.zeros([ns,n]))) # This is the posterior of parameter, which is an empirical distribution with ns samples

In [6]:
# Data
true_mu = np.ones(n) # The true value of the mean
x_data = true_mu + np.random.randn(nobs,n) # Observations: multivariate normal distribution
print("True mean:", true_mu)

('True mean:', array([1., 1., 1., 1., 1.]))


In [4]:
# Inference by MCMC-Gibbs sampling
inference = ed.Gibbs({mu: qmu}, data={x: x_data})
inference.initialize(n_iter=1000)
tf.global_variables_initializer().run()

for _ in range(inference.n_iter):
    info_dict = inference.update()
    inference.print_progress(info_dict)

inference.finalize()

  10/1000 [  1%]                                ETA: 13s | Acceptance Rate: 1.000

/anaconda2/lib/python2.7/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \


1000/1000 [100%] ██████████████████████████████ Elapsed: 6s | Acceptance Rate: 1.001


In [5]:
# View estimations
sess = ed.get_session()
samples_mu = [sess.run(qmu) for i in range(1000)]
muest = np.mean(samples_mu, axis=0)
rmse = np.sqrt(np.mean([(np.linalg.norm(samples_mu[i]-true_mu))**2 for i in range(len(samples_mu))]))
print("Estimated mean: ", muest)
print("RMSE: ", rmse)

('Estimated mean: ', array([1.0053271, 1.0013295, 0.9984345, 1.0063543, 0.9976095],
      dtype=float32))
('RMSE: ', 0.01572366917193939)
